# Setup

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import deeplenstronomy.deeplenstronomy as dl
from deeplenstronomy.visualize import view_image, view_image_rgb

import astropy
from astropy.io import fits
from astropy.table import Table
from astropy.visualization import make_lupton_rgb
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import configparser
import psycopg2
import re
from datetime import datetime
import scipy
import random

pd.set_option('display.max_columns', None)

bands = ['G', 'R', 'Z']

# Training set configs

In [5]:
# Original configs used for model
import yaml

# Create config YAML file
configs = []

configs.append(
    {'DATASET': [
        {'NAME': 'Shrihan_DELVE'},
        {'PARAMETERS': [
            {'SIZE': 80000},
            {'OUTDIR': 'delve_galgal_dataset'},
            {'SEED': 1738},
        ]},
    ]}
)

configs.append(
    {'BACKGROUNDS': [
        {'PATH': 'sim/backgrounds'},
        {'CONFIGURATIONS': '[{config_list}]'},  # Will fill this with e.g. ['CONFIGURATION_1', 'CONFIGURATION_2']
        {'ITERATE': 1}
    ]}
)

configs.append(
    {'DISTRIBUTIONS': [
        {'USERDIST_2': [
            {'FILENAME': 'sim/userdist/source_colors_with_zlens.txt'},
            {'MODE': 'sample'},
        ]},
    ]}
)

configs.append(
    {'COSMOLOGY': [
        {'PARAMETERS': [
            {'H0': 70},
            {'Om0':0.3},
        ]},
    ]}
)

configs.append(
    {'IMAGE': [
        {'PARAMETERS': [
            {'exposure_time': [  # Going to overwrite this with userdist
                {'DISTRIBUTION': [
                    {'NAME': 'delve_exposure_time'},
                    {'PARAMETERS': 'None'},
                ]}
            ]},
            {'numPix': 45},
            {'pixel_scale': 0.2636},
            {'psf_type': 'GAUSSIAN'},
            {'read_noise': 0},  # Set to zero - background noise already included by background images
            {'ccd_gain': [
                {'DISTRIBUTION': [
                    {'NAME': 'des_ccd_gain'},
                    {'PARAMETERS': 'None'},
                ]},
            ]},
        ]},
    ]}
)

configs.append(
    {'SURVEY': [
        {'PARAMETERS': [
            {'BANDS': 'g,r,i,z'},
            {'seeing': [
                {'DISTRIBUTION': [
                    {'NAME': 'delve_seeing'},
                    {'PARAMETERS': 'None'},
                ]},
            ]},
            {'magnitude_zero_point': [  # Going to overwrite this with userdist
                {'DISTRIBUTION': [
                    {'NAME': 'delve_magnitude_zero_point'},
                    {'PARAMETERS': 'None'},
                ]},
            ]},
            {'sky_brightness': 0},  # Set to zero - background noise already included by background images
            {'num_exposures': 1},
        ]},
    ]}
)

configs.append(
    {'SPECIES': [
        {'GALAXY_1': [
            {'NAME': 'DARKLENS'},
            {'LIGHT_PROFILE_1': [
                {'NAME': 'SERSIC_ELLIPSE'},
                {'PARAMETERS': [
                    # Don't think any of this matters, just need to enter valid numbers
                    {'magnitude': 1000},
                    {'center_x': 0.0},
                    {'center_y': 0.0},
                    {'R_sersic': 6.0},
                    {'n_sersic': 4.0},
                    {'e1': -0.5},
                    {'e2': 0.5},
                ]},
            ]},
            {'MASS_PROFILE_1': [
                {'NAME': 'SIE'},
                {'PARAMETERS': [
                    # This is actually sampled
                    {'theta_E': [
                        {'DISTRIBUTION': [
                            {'NAME': 'lognormal'},
                            {'PARAMETERS': [
                                {'mean': 0.91629},  # ln(2.5)
                                {'sigma': 0.405465},  # ln(1.5)
                            ]},
                        ]},
                    ]},
                    {'e1': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'e2': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'center_x': 0.0},  # Lens galaxy should be at center of image in background images
                    {'center_y': 0.0},
                ]},
            ]},
            {'SHEAR_PROFILE_1': [
                {'NAME': 'SHEAR'},
                {'PARAMETERS': [
                    {'gamma1': [
                        {'DISTRIBUTION': [
                            {'NAME': 'lognormal'},
                            {'PARAMETERS': [
                                {'mean': -3.5065579},  # ln(0.03)
                                {'sigma': 0.621698},  # ln(10^.27)
                            ]},
                        ]},
                    ]},
                    {'gamma2': [
                        {'DISTRIBUTION': [
                            {'NAME': 'lognormal'},
                            {'PARAMETERS': [
                                {'mean': -3.5065579},  # ln(0.03)
                                {'sigma': 0.621698},  # ln(10^.27)
                            ]},
                        ]},
                    ]},
                ]},
            ]},
        ]},
        {'GALAXY_2': [
            {'NAME': 'SOURCE'},
            {'LIGHT_PROFILE_1': [
                {'NAME': 'SERSIC_ELLIPSE'},
                {'PARAMETERS': [
                    {'magnitude': 17},  # We're going to overwrite this with a USERDIST
                    {'center_x': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -1.25},
                                {'maximum': 1.25},
                            ]},
                        ]},
                    ]},
                    {'center_y': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -1.25},
                                {'maximum': 1.25},
                            ]},
                        ]},
                    ]},
                    {'R_sersic': [
                        {'DISTRIBUTION': [
                            {'NAME': 'lognormal'},
                            {'PARAMETERS': [
                                {'mean': -0.693147},  # ln(0.5)
                                {'sigma': 0.693147},  # ln(2.0)
                            ]},
                        ]},
                    ]},
                    {'n_sersic': [
                        {'DISTRIBUTION': [
                            {'NAME': 'lognormal'},
                            {'PARAMETERS': [
                                {'mean': 0.22314},  # ln(1.25)
                                {'sigma': 0.81093},  # ln(2.25)
                            ]},
                        ]},
                    ]},
                    {'e1': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'e2': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                ]},
            ]},
            {'MASS_PROFILE_1': [
                {'NAME': 'SIE'},
                {'PARAMETERS': [
                    # Not used, but required?
                    {'sigma_v': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': 200.0},
                                {'maximum': 500.0},
                            ]},
                        ]},
                    ]},
                    {'e1': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'e2': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'center_x': 0.0},  # Galaxy should be at center of image in background images
                    {'center_y': 0.0},
                ]},
            ]},
        ]},
        {'GALAXY_3': [
            {'NAME': 'SIMLENS'},
            {'LIGHT_PROFILE_1': [
                {'NAME': 'SERSIC_ELLIPSE'},
                {'PARAMETERS': [
                    {'magnitude': 17},  # We're going to overwrite this with a USERDIST
                    {'center_x': 0.0},
                    {'center_y': 0.0},
                    {'R_sersic': [
                        {'DISTRIBUTION': [
                            {'NAME': 'normal'},
                            {'PARAMETERS': [
                                {'mean': 6.0},
                                {'std': 1.5},
                            ]},
                        ]},
                    ]},
                    {'n_sersic': [
                        {'DISTRIBUTION': [
                            {'NAME': 'normal'},
                            {'PARAMETERS': [
                                {'mean': 4.0},
                                {'std': 0.8},
                            ]},
                        ]},
                    ]},
                    {'e1': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'e2': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                ]},
            ]},
            {'MASS_PROFILE_1': [
                {'NAME': 'SIE'},
                {'PARAMETERS': [
                    # Not used, but required?
                    {'sigma_v': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': 150.0},
                                {'maximum': 450.0},
                            ]},
                        ]},
                    ]},
                    {'e1': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'e2': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': -0.5},
                                {'maximum': 0.5},
                            ]},
                        ]},
                    ]},
                    {'center_x': 0.0},  # Galaxy should be at center of image in background images
                    {'center_y': 0.0},
                ]},
            ]},
        ]},
    ]}
)

configs.append(
    {'GEOMETRY': [
        {'CONFIGURATION_1': [
            {'NAME': 'LENSES'},
            {'FRACTION': 0.5},
            {'PLANE_1': [
                {'OBJECT_1': 'DARKLENS'},
                {'PARAMETERS': [
                    {'REDSHIFT': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': 0.45},
                                {'maximum': 0.7499},
                            ]},
                        ]},
                    ]},
                ]},
            ]},
            {'PLANE_2': [
                {'OBJECT_1': 'SOURCE'},
                {'PARAMETERS': [
                    {'REDSHIFT': [  # We're going to overwrite this with a USERDIST
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': 1.0},
                                {'maximum': 2.0},
                            ]},
                        ]},
                    ]},
                ]},
            ]},
        ]},
        {'CONFIGURATION_2': [
            {'NAME': 'NONLENSES'},
            {'FRACTION': 0.5},
            {'PLANE_1': [
                {'OBJECT_1': 'DARKLENS'},
                {'PARAMETERS': [
                    {'REDSHIFT': [  # We're going to overwrite this with a USERDIST
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': 0.45},
                                {'maximum': 0.7499},
                            ]},
                        ]},
                    ]},
                ]},
            ]},
            {'PLANE_2': [
                {'OBJECT_1': 'DARKLENS'},
                {'PARAMETERS': [
                    {'REDSHIFT': [
                        {'DISTRIBUTION': [
                            {'NAME': 'uniform'},
                            {'PARAMETERS': [
                                {'minimum': 1.0},
                                {'maximum': 2.0},
                            ]},
                        ]},
                    ]},
                ]},
            ]},
        ]},
    ]},
)

# Dump yaml
outpath = 'configs/delve_galgal.yaml'
with open(outpath, 'w') as f:
    upload = yaml.dump(configs, f)

# Reformat as needed
with open(outpath, 'r') as f:
    text = f.read()
text = text.format(config_list="'CONFIGURATION_1', 'CONFIGURATION_2'")
text = text.replace("'[", '[')
text = text.replace("]'", ']')
text = text.replace('- ', '')
with open(outpath, 'w') as f:
    f.write(text)

5791

In [60]:
udist = pd.read_csv('sim/userdist/source_colors_with_zlens.txt', sep='\s')
udist.shape
udist.head()

(7594, 7)

,CONFIGURATION_1-PLANE_2-OBJECT_1-REDSHIFT,CONFIGURATION_1-PLANE_2-OBJECT_1-LIGHT_PROFILE_1-magnitude-g,CONFIGURATION_1-PLANE_2-OBJECT_1-LIGHT_PROFILE_1-magnitude-r,CONFIGURATION_1-PLANE_2-OBJECT_1-LIGHT_PROFILE_1-magnitude-i,CONFIGURATION_1-PLANE_2-OBJECT_1-LIGHT_PROFILE_1-magnitude-z,CONFIGURATION_1-PLANE_1-OBJECT_1-REDSHIFT,WEIGHT
0,1.493197,21.875757,21.723236,21.385426,20.283205,0.280203,1.0
1,1.442902,21.044741,22.096289,22.480694,21.414358,0.691930,1.0
2,1.430981,21.641100,21.675371,21.582199,21.397528,0.685003,1.0
3,1.561445,21.906784,21.832739,21.321911,19.981070,0.589351,1.0
4,1.305264,20.681000,20.718277,20.033199,19.972401,0.327502,1.0


## Simulate

In [137]:
config_file = 'configs/delve_galgal.yaml'

In [2]:
! cat configs/delve_galgal.yaml

DATASET:
  NAME: DELVE_GalGal
  PARAMETERS:
    SIZE: 80000
    OUTDIR: delve_galgal_dataset_full_sample
    SEED: 1738
BACKGROUNDS:
  PATH: /home/s1/ezaborow/projects/strong_lensing/analysis/cutouts/train_imgs/2021/full_sample/cutouts_deeplenstronomy
  CONFIGURATIONS: ['CONFIGURATION_1', 'CONFIGURATION_2']
  ITERATE: 1
DISTRIBUTIONS:
  USERDIST_2:
    FILENAME: /home/s1/ezaborow/projects/strong_lensing/analysis/new_model/sim_training_set/2021/configs/source_colors_config_1_wide_with_zlens.txt
    MODE: sample
COSMOLOGY:
  PARAMETERS:
    H0: 70
    Om0: 0.3
IMAGE:
  PARAMETERS:
    exposure_time:
      DISTRIBUTION:
        NAME: delve_exposure_time
        PARAMETERS: None
    numPix: 45
    pixel_scale: 0.2636
    psf_type: GAUSSIAN
    read_noise: 0
    ccd_gain:
      DISTRIBUTION:
        NAME: des_ccd_gain
        PARAMETERS: None
SURVEY:
  PARAMETERS:
    BANDS: g,r,i,z
    seeing:
      DISTRIBUTION:
        NAME: delve_seeing
        PARAMETERS: None
    magnitude_zero_point:

In [142]:
dataset = dl.make_dataset(config_file, verbose=True, save_to_disk=True)

Entering main organization loop
Organizing CONFIGURATION_1
Organizing CONFIGURATION_2
Generating images for CONFIGURATION_1
	Progress: 100.0 %  ---  Elapsed Time: 0 H 0 M 3 S         
Generating images for CONFIGURATION_2
	Progress: 100.0 %  ---  Elapsed Time: 0 H 0 M 3 S         


The image arrays have been stored as numpy files. They can be loaded by doing
```python
images = numpy.load('DES_GalGal/CONFIGURATION_1_images.npy')
```

The metadata dataframes have been written to csv files. They can be loaded by doing
```python
metadata = pandas.read_csv('DES_GalGal/CONFIGURATION_1_metadata.csv')
```

Future versions of `lenspy` will include file format flexibility and built in dataset loading funcitons.